In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *
from fastai.text import *
from fastai.column_data import *

In [2]:
DATA_PATH = Path('data')
DATA_PATH
TMP_PATH = DATA_PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)

In [ ]:
TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

In [ ]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

In [3]:
sql_ids = np.load(TMP_PATH/'sql_ids.npy')
sql_itos = pickle.load(open(TMP_PATH/'sql_itos.pkl','rb'))
sql_stoi =  collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(sql_itos)})

In [4]:
def get_next_random(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    p = to_np(p)[0]
    pi = p.argsort()[-10:]
    i = np.random.choice(pi, p=softmax_np(p[pi]))
    return indices_char[i]
def softmax_np (nparray):
    ans = np.exp(nparray)
    return ans/ans.sum()

def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    p = to_np(p)
    
    return indices_char[i]

def get_sentence(inp):
    sentence = inp
    counter = 0
    tmp = []
    while "_eos_" not in tmp:   
        tmp = get_next(sentence[-8:]) # always get last three
        sentence += [tmp]
        counter = counter+1
        if counter> 500:
            break
    return sentence
        

def get_sentence_random(inp):
    sentence = inp
    counter = 0
    tmp = []
    while "_eos_" not in tmp:   
        tmp = get_next_random(sentence[-8:]) # always get last three
        sentence += [tmp]
        counter = counter+1
        if counter> 500:
            break
    return sentence
        

def show_sentence(inp):
    sentence =''
    captialize = False
    for i in inp:
        if i =='t_up':
            captialize = True
        else:            
            if captialize:
                sentence = sentence + " " + i.upper()       
           
            else:
                sentence = sentence +" " + i
            captialize = False
    return sentence

In [5]:
def get_next_random_fsm(inp, fsm):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    p = to_np(p)[0]
    pi = p.argsort()[-10:]
    i = np.random.choice(pi, p=softmax_np(p[pi]))
    
    if fsm.run(indices_char[i]):
        return indice_char[i]
    else: 
        for pii in pi:
            if fsm.run(indice_char[pii]):
                print('pii loop')
                return indice_char[pii]
            else:
                print('Error, no valid words at all')  
    
    
    return "ERROR"

In [9]:
fsm = Rule_Parse_FSM()

In [10]:
get_next_random("select userid from users where ".split())

NameError: name 'char_indices' is not defined

In [ ]:
get_next_random("select userid from users where ".split())

In [ ]:
get_next_random("select userid from users where ".split())

In [ ]:
get_next_random("select userid from users where ".split())

In [50]:
get_next_random("select userid from users where ".split())

'tags'

In [49]:
get_next_random("select userid from users where ".split())

'posts'

In [48]:
get_next_random_fsm("select userid from users where ".split(), fsm)

skip 'location' in STATE: NEW_GROUP


NameError: name 'indice_char' is not defined

## Setup

Set up directory path

In [4]:
DATA_PATH = Path('data')
DATA_PATH
TMP_PATH = DATA_PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)

In [9]:
dataset = pd.read_json(DATA_PATH/'train.json')
dataset.head()


,comments,description,id,sql,sql_plain,title,url
0,[],,586912,"SELECT p.Title,p.score,p.ViewCount,p.AnswerCou...","SELECT \np.Title,p.score,p.ViewCount,p.AnswerC...",Repent 151-160,http://data.stackexchange.com/stackoverflow/re...
1,[This is test data for my question to make thi...,,54791,"CREATE TABLE #CustomerDB ( ID int NOT NULL, NA...",-- This is test data for my question to make t...,This is test data for my question to make thin...,http://data.stackexchange.com/stackoverflow/re...
2,[],,611565,select Id from Comments where lower(Text) like...,select Id\nfrom Comments \nwhere lower(Text) l...,Accept answer,http://data.stackexchange.com/stackoverflow/re...
3,[],,612812,"DECLARE @From DATETIME = convert(DATETIME, '##...","DECLARE @From DATETIME = convert(DATETIME, '##...",Questions with most views created within 3 mon...,http://data.stackexchange.com/stackoverflow/re...
4,"[Most Prolific Answerers (with score), Shows t...",,325114,"SELECT TOP 50 OwnerUserId as [User Link], COUN...",-- Most Prolific Answerers (with score)\n-- Sh...,Most Prolific Answers,http://data.stackexchange.com/stackoverflow/re...


In [10]:
SQL = dataset.sql

In [11]:
## Transform the text to be tokenize readily, as certain token has special meaning in SQL, ie. "." means the schema relation
token_replaced = {',' : ' , ',
                 '#' : ' # ',
                 '@' : ' @ ',
                 '!' : ' ! ',
                 '.' : ' . ',
                 '%' : ' % ',
                 '?' : ' ? ',
                 ')' : ' ) ',
                 '(' : ' ( ',
                 '=' : ' = ',
                 '/' : ' / ',
                 '*' : ' * '}
    

In [15]:
def fixup(x):
    for bereplaced, replace in token_replaced.items():        
        x = x.replace(bereplaced, replace).lower() # CASE insentive for SQL
    return x

In [16]:
SQL = SQL.apply(fixup)## Sanity Check
SQL.iloc[0]

'select p . title , p . score , p . viewcount , p . answercount , p . commentcount , len ( p . body )  as ques_size ,  p . favoritecount , p . id ,  u . reputation as user_repo , u . creationdate as user_join_date ,  datediff ( minute ,  p . creationdate ,  a . creationdate )  as qatimegap from posts as p ,  posts as a ,  users as u where p . id = a . parentid and p . owneruserid = u . id and a . creationdate  =   ( select min ( tau . creationdate )  from posts tau where tau . parentid = p . id group by tau . parentid )  and  (  p . title like \' % recommended method for handling unsupportedencodingexception from string . getbytes ( "utf-8" )  % \' or p . title like \' % way to format strings with " ? " parameters to full string in java ?  % \' or p . title like \' % how to parse a string without regular expressions % \' or p . title like \' % how to replace string only once without regex in java ?  % \' or p . title like \' % efficiently removing specific characters  ( some punctuatio

In [17]:
SQL.iloc[0]

'select p . title , p . score , p . viewcount , p . answercount , p . commentcount , len ( p . body )  as ques_size ,  p . favoritecount , p . id ,  u . reputation as user_repo , u . creationdate as user_join_date ,  datediff ( minute ,  p . creationdate ,  a . creationdate )  as qatimegap from posts as p ,  posts as a ,  users as u where p . id = a . parentid and p . owneruserid = u . id and a . creationdate  =   ( select min ( tau . creationdate )  from posts tau where tau . parentid = p . id group by tau . parentid )  and  (  p . title like \' % recommended method for handling unsupportedencodingexception from string . getbytes ( "utf-8" )  % \' or p . title like \' % way to format strings with " ? " parameters to full string in java ?  % \' or p . title like \' % how to parse a string without regular expressions % \' or p . title like \' % how to replace string only once without regex in java ?  % \' or p . title like \' % efficiently removing specific characters  ( some punctuatio

In [4]:
sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL)) 

NameError: name 'SQL' is not defined

In [19]:
sql_tok[0]

['select',
 'p',
 '.',
 'title',
 ',',
 'p',
 '.',
 'score',
 ',',
 'p',
 '.',
 'viewcount',
 ',',
 'p',
 '.',
 'answercount',
 ',',
 'p',
 '.',
 'commentcount',
 ',',
 'len',
 '(',
 'p',
 '.',
 'body',
 ')',
 'as',
 'ques_size',
 ',',
 'p',
 '.',
 'favoritecount',
 ',',
 'p',
 '.',
 'i',
 'd',
 ',',
 'u',
 '.',
 'reputation',
 'as',
 'user_repo',
 ',',
 'u',
 '.',
 'creationdate',
 'as',
 'user_join_date',
 ',',
 'datediff',
 '(',
 'minute',
 ',',
 'p',
 '.',
 'creationdate',
 ',',
 'a',
 '.',
 'creationdate',
 ')',
 'as',
 'qatimegap',
 'from',
 'posts',
 'as',
 'p',
 ',',
 'posts',
 'as',
 'a',
 ',',
 'users',
 'as',
 'u',
 'where',
 'p',
 '.',
 'i',
 'd',
 '=',
 'a',
 '.',
 'parentid',
 'and',
 'p',
 '.',
 'owneruserid',
 '=',
 'u',
 '.',
 'i',
 'd',
 'and',
 'a',
 '.',
 'creationdate',
 '=',
 '(',
 'select',
 'min',
 '(',
 'tau',
 '.',
 'creationdate',
 ')',
 'from',
 'posts',
 'tau',
 'where',
 'tau',
 '.',
 'parentid',
 '=',
 'p',
 '.',
 'i',
 'd',
 'group',
 'by',
 'tau',
 '.',

In [20]:
def toks2ids(tok,pre):
    freq = Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common()]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    ids = np.array([([0] + [stoi[o] for o in p] + [2]) for p in tok]) # pad bos at beginning and eos at the end
    np.save(TMP_PATH/f'{pre}_ids.npy', ids)
    pickle.dump(itos, open(TMP_PATH/f'{pre}_itos.pkl', 'wb'))
    return ids,itos,stoi

In [21]:
sql_ids,sql_itos,sql_stoi = toks2ids(sql_tok,'sql')

In [12]:
sql_ids_concat= []
for i in sql_ids:
    sql_ids_concat += i

In [13]:
sql_ids.shape

(24594,)

In [14]:
vocab_size = len(sql_itos)
print('total chars:', vocab_size)

total chars: 23125


### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [15]:
idx = sql_ids_concat

In [16]:
char_indices = sql_stoi
indices_char = sql_itos

### Create and train model

Pick a size for our hidden state

In [17]:
n_hidden = 256
n_fac = 42

## Wrap Char3Model to fastai learner

### Create inputs

In [18]:
cs=5

In [19]:
c_in_dat = [[sql_ids_concat[i+j] for i in range(cs)] for j in range(len(sql_ids_concat)-cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [20]:
c_out_dat = [sql_ids_concat[j+cs] for j in range(len(sql_ids_concat)-cs)]

In [21]:
xs = np.stack(c_in_dat, axis=0)

In [22]:
xs

array([[  0,  13,  18,   4, 102],
       [ 13,  18,   4, 102,   5],
       [ 18,   4, 102,   5,  18],
       ...,
       [ 27,  20,   8, 252,  30],
       [ 20,   8, 252,  30,   8],
       [  8, 252,  30,   8, 124]])

In [23]:
xs.shape

(2146779, 5)

In [24]:
y = np.stack(c_out_dat)

In [25]:
y

array([  5,  18,   4, ...,   8, 124,   2])

So each column below is one series of 8 characters from the text.

In [26]:
xs[:cs,:cs]

array([[  0,  13,  18,   4, 102],
       [ 13,  18,   4, 102,   5],
       [ 18,   4, 102,   5,  18],
       [  4, 102,   5,  18,   4],
       [102,   5,  18,   4,  33]])

...and this is the next character after each sequence.

In [27]:
y[:cs]

array([ 5, 18,  4, 33,  5])

### Create and train model

In [28]:
val_idx = get_cv_idxs(len(sql_ids_concat)-cs-1)

In [29]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=2048)

In [30]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)          
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [31]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
model = RNN_Learner(md, SingleModel(to_gpu(m)), opt_fn=opt_fn)


C:\Users\NokChan\Anaconda3\envs\fastai\lib\site-packages\torch\cuda\__init__.py:116: UserWarning: 
    Found GPU0 GeForce 930M which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [25]:
model.fit(1e-3,1, cycle_len=1,use_clr_beta = (10,10,0.95,0.85))

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      3.165478   3.135849  



[3.1358491721754476]

In [179]:
a=get_sentence(['select','*','from','where','users'])

In [180]:
b=get_sentence_random(['select','*','from','where','users'])

In [2]:
show_sentence(b)

NameError: name 'show_sentence' is not defined

In [182]:
show_sentence(a)

' select * from where users . i d = # # userid # # and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p . posttypeid = 1 and p 

In [138]:
show_sentence(a)

" SELECT * FROM where customer LIKE ' % < % ' or LOWER ( location ) LIKE ' % india % ' ORDER by reputation DESC ; _eos_"

In [ ]:
get_next(['select', '*', 'from'])

### Test model

In [98]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [99]:
get_next('for thos')

'e'

In [100]:
get_next('part of ')

't'

In [101]:
get_next('queens a')

'n'

In [118]:
get_next('for thos')

'e'

In [119]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [120]:
get_next_n('for thos', 40)

'for those the same the same the same the same th'

## Multi-output model

In [19]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

In [20]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [21]:
xs = np.stack(c_in_dat)
xs.shape

(75111, 8)

In [22]:
ys = np.stack(c_out_dat)
ys.shape

(75111, 8)

In [23]:
xs[:10,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [ 1,  1, 43, 45, 40, 40, 39, 43],
       [33, 38, 31,  2, 73, 61, 54, 73],
       [ 2, 44, 71, 74, 73, 61,  2, 62],
       [72,  2, 54,  2, 76, 68, 66, 54],
       [67,  9,  9, 76, 61, 54, 73,  2],
       [73, 61, 58, 67, 24,  2, 33, 72],
       [ 2, 73, 61, 58, 71, 58,  2, 67]])

In [24]:
ys[:10,:cs]

array([[42, 29, 30, 25, 27, 29,  1,  1],
       [ 1, 43, 45, 40, 40, 39, 43, 33],
       [38, 31,  2, 73, 61, 54, 73,  2],
       [44, 71, 74, 73, 61,  2, 62, 72],
       [ 2, 54,  2, 76, 68, 66, 54, 67],
       [ 9,  9, 76, 61, 54, 73,  2, 73],
       [61, 58, 67, 24,  2, 33, 72,  2],
       [73, 61, 58, 71, 58,  2, 67, 68]])

### Create and train model

In [127]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [128]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

### Identity init!

In [140]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [141]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [142]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       2.39428  2.21111]                                
[ 1.       2.10381  2.03275]                                
[ 2.       1.99451  1.96393]                               
[ 3.       1.93492  1.91763]                                



In [143]:
set_lrs(opt, 1e-3)

In [144]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       1.84035  1.85742]                                
[ 1.       1.82896  1.84887]                                
[ 2.       1.81879  1.84281]                               
[ 3.       1.81337  1.83801]                                



## Stateful model

### Setup

### GRU

In [11]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [12]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512).cuda()

opt = optim.Adam(m.parameters(), 1e-3)

NameError: name 'md' is not defined

In [29]:
fit(m, md, 6, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.68409  1.67784]                                 
[ 1.       1.49813  1.52661]                                 
[ 2.       1.41674  1.46769]                                 
[ 3.       1.36359  1.43818]                                 
[ 4.       1.33223  1.41777]                                 
[ 5.       1.30217  1.40511]                                 



In [30]:
set_lrs(opt, 1e-4)

In [31]:
fit(m, md, 3, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.22708  1.36926]                                 
[ 1.       1.21948  1.3696 ]                                 
[ 2.       1.22541  1.36969]                                 



In [18]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [19]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.72032  1.64016]                                 
[ 1.       1.62891  1.58176]                                 



In [8]:
import re
import pdb

def transition_skip(fsm_obj):
    pass


def transition_new_group(fsm_obj):
    fsm_obj.group_current_level += 1
    fsm_obj.current_group = RuleGroup(fsm_obj.current_group,
                                      fsm_obj.group_current_level,
                                      None)


def transition_append_pre(fsm_obj):
    rule_count = fsm_obj.current_group.rule_count
    fsm_obj.current_group.rules[rule_count - 1].prefix += fsm_obj.current_char


def transition_add_op(fsm_obj):
    rule_count = fsm_obj.current_group.rule_count
    fsm_obj.current_group.rules[rule_count - 1].op = fsm_obj.current_char


def transition_end_group(fsm_obj):
    fsm_obj.group_current_level += 1
    fsm_obj.current_group = RuleGroup(fsm_obj.current_group,
                                      fsm_obj.group_current_level,
                                      None)


def transition_end_rule(fsm_obj):
    pass


def transition_add_op_new_rule(fsm_obj):
    fsm_obj.current_group.rule_count += 1
    fsm_obj.current_group.rules.append(Rule())
    rule_count = fsm_obj.current_group.rule_count
    fsm_obj.current_group.rules[rule_count - 1].op = fsm_obj.current_char


def transition_append_subj(fsm_obj):
    rule_count = fsm_obj.current_group.rule_count
    fsm_obj.current_group.rules[rule_count - 1].subject += fsm_obj.current_char


def transition_add_op_new_group(fsm_obj):
    fsm_obj.current_group.op = fsm_obj.current_char


T_SKIP = transition_skip
T_NEW_GROUP = transition_new_group
T_APPEND_CHAR_PRE = transition_append_pre
T_ADD_OP = transition_add_op
T_ADD_OP_NEW_RULE = transition_add_op_new_rule
T_END_GROUP = transition_end_group
T_END_RULE = transition_end_rule
T_APPEND_CHAR_SUBJ = transition_append_subj
T_ADD_GROUP_OP = transition_add_op_new_group

S_NEW_GROUP = "STATE: NEW_GROUP"
S_END_GROUP = "STATE: END_GROUP"
S_PRE = "STATE: PREFIX"
S_OP = "STATE: OPERATOR"
S_END_RULE = "STATE: END_RULE"
S_SUBJ = "STATE: SUBJECT"


FSM_MAP = (
    #  {'src':, 'dst':, 'condition':, 'callback': },
    {'src': S_NEW_GROUP,
        'dst': S_PRE,
        'condition': r"\b(select)\b",
        'callback': T_APPEND_CHAR_PRE},  # 1
    {'src': S_PRE,
        'dst': S_SUBJ,
        'condition': r"[A-Za-z]+",
        'callback': T_APPEND_CHAR_PRE},  # 2
    {'src': S_SUBJ,
        'dst': S_END_RULE,
        'condition': r"\b(from)\b",
        'callback': T_APPEND_CHAR_PRE},  # 3
    {'src': S_END_RULE,
        'dst': S_NEW_GROUP,
        'condition': r"[A-Za-z]+",
        'callback': T_SKIP} ) # 4    




for map_item in FSM_MAP:
    map_item['condition_re_compiled'] = re.compile(map_item['condition'])


class Rule:
    def __init__(self):
        self.prefix = ""
        self.subject = ""
        self.op = None

    def __repr__(self):
        op = self.op
        if not op:
            op = ''
        return "<Rule: {} {}({})>".format(op, self.prefix, self.subject)

class RuleGroup:
    def __init__(self, parent, level, op):
        self.op = op
        self.parent = parent
        self.level = level
        self.rule_count = 1
        self.rules = [Rule(), ]

    def __repr__(self):
        return "<RuleGroup: {}>".format(self.__dict__)


class Rule_Parse_FSM:

    def __init__(self, input_word=""):
        self.input_word = input_word
        self.current_state = S_NEW_GROUP
        self.group_current_level = 0
        self.current_group = RuleGroup(None, self.group_current_level, None)
        self.current_char = ''
        self.valid_word = False

    def run(self,inp):
        self.input_word = inp
        if not self.process_next(self.input_word):
            print("skip '{}' in {}".format(self.input_word, self.current_state))
            self.valid_word = False
        else: self.valid_word = True
        return self.valid_word

                
    def process_next(self, achar):
        self.current_char = achar
        frozen_state = self.current_state
        # pdb.set_trace()
        for transition in FSM_MAP:
            if transition['src'] == frozen_state:
                if self.iterate_re_evaluators(achar, transition):
                    return True
        return False

    def iterate_re_evaluators(self, achar, transition):
        condition = transition['condition_re_compiled']
        # pdb.set_trace()
        if condition.match(achar):
            self.update_state(
                transition['dst'], transition['callback'])
            return True
        return False

    def update_state(self, new_state, callback):
        print("{} -> {} : {}".format(self.current_char,
                                     self.current_state,
                                     new_state))
        self.current_state = new_state
        callback(self)